# 크롬드라이버 설정 방법 - Mac (2023 Aug)

## 개요
- `selenium 4.10` 버전에서 크롬드라이버 설정

## selenium 버전 확인
- 먼저 selenium 버전은 아래와 같다.
- 설치 방법
```python
pip install selenium

```

In [1]:
import selenium
print(selenium.__version__)

4.11.0


## 크롬드라이버 수동 설정
- 특별한 경로에 chromedriver.exe 파일을 설치한다. (경로 기억, 주의)

In [2]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

list_files("driver")

driver/
    chromedriver
    LICENSE.chromedriver
    .ipynb_checkpoints/
        LICENSE-checkpoint.chromedriver


## 📍주의 : 셀레늄 버전 확인
- 셀레늄 버전이 4.3이후로 완전히 바뀌었음
- 문법(메서드)도 많이 바뀜 > 게시글은 2023년 이후 것만 구글링으로 확인

- 이 방식은 하나의 치명적인 단점이 존재한다. 위의 코드는 매우 잘 작동하지만 Chrome이 새 버전으로 업그레이드될 때마다 ChromeDriver를 다시 다운로드해야 한다.
- 매우 불편하다. 

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time

CHROME_DRIVER_PATH = './driver/chromedriver'
service = Service(executable_path=CHROME_DRIVER_PATH)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.google.co.kr/')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [4]:
# 드라이버 종료
driver.quit()

## webdriver-manager 라이브러리
- 이 부분을 해결하고자 라이브러리가 나왔다.
```python
pip install webdriver-manager

```
- 코드는 간단하다. 

In [7]:
# 아래 네 줄을 계속 활용하면 됨!
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
#driver.get('https://www.naver.com/')

## 1. 구글이미지 다운로드

In [20]:
#구글이미지 다운로드

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

url = 'https://www.google.co.kr/imghp'
driver.get(url)

elem = driver.find_element(By.CSS_SELECTOR, "body > div.L3eUgb > div.o3j99.ikrT4e.om7nvf > form > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > textarea.gLFyf")
elem.send_keys("보라카이")    #구글 검색창에 보라카이 입력
elem.send_keys(Keys.RETURN) #엔터!

# 스크롤내리며 이미지 쭉 확인하기
elem = driver.find_element(By.TAG_NAME, "body")
for i in range(60):
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)

# 이미지 클릭하기
try:
    driver.find_element(By.CSS_SELECTOR, "islmp > div > div > div > div.gBPM8 > div.qvfT1 > div.YstHxe > input").click()
    for i in range(60):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.1)
except:
    pass

links=[] 
images = driver.find_elements(By.CSS_SELECTOR, "#islrg > div.islrc > div > a.wXeWr.islib.nfEiy > div.bRMDJf.islir > img")

for image in images:
    if image.get_attribute('src') is not None:
        links.append(image.get_attribute('src'))
        
print(' 찾은 이미지 개수:',len(links))

 찾은 이미지 개수: 48


In [23]:
import urllib.request

for k, i in enumerate(links):
    url = i
    urllib.request.urlretrieve(url, "./images/"+str(k)+".jpg")

print('다운로드 완료하였습니다.')

다운로드 완료하였습니다.


In [25]:
driver.quit()

## 2. 검색어 추출

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = "https://signal.bz/news"
driver.get(url)

results = driver.find_elements(By.CSS_SELECTOR, "#app > div > main > div > section > div > section > section:nth-child(2) > div:nth-child(2) > div > div > div > a > span.rank-text")
results

keyword_list = []
for keyword in results:
    keyword_list.append(keyword.text)

keyword_list

['잼버리',
 '살인 예고 글',
 '태풍 카눈 북상에',
 '류현진',
 '세계 고양이의 날',
 '파두',
 '경상수지 58.7억달러',
 '최원종',
 '입추',
 '초전도체']

In [33]:
driver.quit()

## 3. 네이트 검색어 1~10위 가져오기(동적화면)

In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

nate_list_1st = []
nate_list_2nd = []
for i in range(2):
    URL='https://www.nate.com'
    driver.get(url=URL) 
    rank_results = driver.find_elements(By.CSS_SELECTOR,'#olLiveIssueKeyword > li > span.num_rank')
    nate_results = driver.find_elements(By.CSS_SELECTOR,'#olLiveIssueKeyword > li > a > span.txt_rank')
    
    for rank, keyword in zip(rank_results, nate_results):
        if i == 0:  #첫번째 화면
            nate_list_1st.append(f'{rank.text}_{keyword.text}')
        elif i == 1: #두번째 화면
            nate_list_2nd.append(f'{rank.text}_{keyword.text}')
            
    time.sleep(5)
    driver.refresh()   #새로고침같은 개념
    
result = nate_list_1st + nate_list_2nd

print(result)
driver.quit()

['1_태풍 카눈 북상에', '2_경상수지 두 달', '3_잼버리', '4_류현진 강습 타구', '5_사고 위장 부사관', '6_잼버리 대원들', '7_서울대공원서 시베리아호랑이 파악', '8_中 윤동주 폐쇄', '9_입추', '10_신성록']


## 뽐뿌 사이트 핫딜조회

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

URL= 'https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url=URL)

titles = driver.find_elements(By.CSS_SELECTOR, "#revolution_main_table > tbody > tr:nth-child(9) > td:nth-child(3) > table > tbody > tr > td:nth-child(2) > div > a > font")
urls = driver.find_elements(By.CSS_SELECTOR, "#revolution_main_table > tbody > tr:nth-child(9) > td:nth-child(3) > table > tbody > tr > td:nth-child(2) > div > a")

title_lists = []
url_lists = []
for i in range(len(titles)):
    print(titles[i].text)
    title_lists.append(titles[i].text)
    print(urls[i].get_attribute('href'))
    url_lists.append(urls[i].get_attribute('href'))

driver.quit()

[티몬] 바세린 퓨어 젤리 50ml 5개 외 (5,520원/무료)
https://www.ppomppu.co.kr/zboard/view.php?id=ppomppu&page=1&divpage=80&no=477026
